#### **Lip Reading Application**

##### Install/Import dependencies

In [ ]:
%%capture
#Install dependencies and confirm installation with !pip list
!pip install opencv-python matplotlib imageio gdown tensorflow-macos
!pip list

In [ ]:
#Import dependencies
import os
import cv2
#import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio
import tensorflow as tf
import gdown

##### Load Data

In [ ]:
%%capture
#Downloading dataset that was made for creating lip models
url = 'https://drive.google.com/uc?id=1YlvpDLix3S-U8fd-gqRwPcWXAXm8JwjL'
output = 'data.zip'
gdown.download(url, output, quiet = False)
gdown.extractall('data.zip')

In [ ]:
def load_video(path) -> List[float]:
    '''
    Takes in a path to a video and returns the float values for each frame
        Args: 
            path: str --> path to video that will be passed into model
        Returns:
            List of floats that represents video1
    '''
    cap = cv2.VideoCapture(path)
    frames=[]
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        success, frame = cap.read()
        frame = tf.image.rgb_to_grayscale(frame)
        #isolates the mouth --> we can also use a lip detector to isolate the mouth
        frames.append(frame[190:236,80:220,:])
    cap.release()
    
    mean = tf.math.reduce_mean(frames)
    std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    return tf.cast((frames-mean), tf.float32)/std                 
    

In [ ]:
vocab=[x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]

In [ ]:
char_num = tf.keras.layers.StringLookup(vocabulary = vocab, oov_token="")
num_char = tf.keras.layers.StringLookup(vocabulary = char_num.get_vocabulary(), oov_token="", invert = True)
print(f'The vocab is: {char_num.get_vocabulary()}'
      f'(size={char_num.vocabulary_size()})')

The vocab is: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'", '?', '!', '1', '2', '3', '4', '5', '6', '7', '8', '9', ' '](size=40)


In [ ]:
def load_alignments(path) -> List[str]:
    with open(path, 'r') as f:
        lines = f.readlines()
    tokens = []
    for line in lines:
        line = line.split()
        if line[2]!='sil':
            tokens=[*tokens,' ', line[2]]
    return char_num(tf.reshape(tf.strings.unicode_split(tokens, input_encoding='UTF-8'), (-1)))[1:]

In [ ]:
def load_data(path):
    path = bytes.decode(path.numpy())
    file_name = path.split('/')[-1].split('.')[0]
    video_path = os.path.join('data','s1',f'{file_name}.mpg')
    alignment_path = os.path.join('data','alignments', 's1',f'{file_name}.align')
    frames = load_video(video_path)
    alignments = load_alignments(alignment_path)
    return frames, alignments

In [ ]:
def mappable_function(path) -> List[str]:
    return tf.py_function(load_data,[path], (tf.float32,tf.int64))